## Diverse_Profiles

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout


# Pfad zur Datensatzdatei
data_path = 'C:/Users/JonasNiehus/Documents/Masterarbeit/Evaluation/Datensatz/german.data'

In [ ]:
# Einlesen des Datensatzes – keine Spaltennamen
df = pd.read_csv(data_path, header=None, sep='\s+')

# Spaltennamen setzen (21 Spalten)
df.columns = [
    "Status_des_Girokontos", "Dauer_in_Monaten", "Kreditgeschichte", "Kreditverwendungszweck",
    "Kreditbetrag", "Sparkonto_Wertpapiere", "Beschäftigt_seit", "Ratenhöhe",
    "Familienstand_Geschlecht", "Weitere_Bürgen_Schuldner", "Wohnsitzdauer", "Vermögen", "Alter",
    "Andere_Ratenverpflichtungen", "Wohnsituation", "Anzahl_bestehender_Kredite", "Beruf",
    "Unterhaltspflichtige_Personen", "Telefon", "Ausländischer_Arbeiter", "Ziel"
]

# Zielvariable umkodieren: 1 = guter Kredit, 2 = schlechter Kredit
df['Ziel'] = df['Ziel'].map({1: 1, 2: 0})  # Umwandlung in 1 und 0 direkt

# Features & Ziel aufteilen
X = df.drop("Ziel", axis=1)  # Alle Features außer der Ziel-Spalte
y = df["Ziel"]  # Ziel ist die 'Ziel'-Spalte

# Manuelle Trennung numerischer/kategorischer Spalten
numerical_cols = ["Dauer_in_Monaten", "Kreditbetrag", "Ratenhöhe", "Wohnsitzdauer", "Alter", "Anzahl_bestehender_Kredite", "Unterhaltspflichtige_Personen"]
categorical_cols = [col for col in X.columns if col not in numerical_cols]

# ColumnTransformer: Skaliert numerisch, encoded kategorisch
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_cols),  # Numerische Features werden skaliert
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)  # Kategorische Features werden one-hot encodiert
])



<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\JonasNiehus\AppData\Local\Temp\ipykernel_9444\368753761.py:2: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(data_path, header=None, sep='\s+')


In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances

def select_diverse_profiles(X_raw, y, preprocessor, n_profiles=10, random_state=42):
    """
    Wählt n möglichst diverse Profile aus dem Datensatz nach dem Max-Min-Prinzip.

    Parameter
    ----------
    X_raw : pd.DataFrame
        Originaldaten (vor Preprocessing, noch mit Kategorien etc.).
    y : pd.Series
        Zielvariable (0 = schlechter Kredit, 1 = guter Kredit).
    preprocessor : ColumnTransformer
        Preprocessing-Pipeline (skalieren + OneHotEncode).
    n_profiles : int
        Anzahl der gewünschten Profile.
    random_state : int
        Zufallsstartwert für Reproduzierbarkeit.

    Returns
    -------
    diverse_profiles : pd.DataFrame
        Ausgewählte Originalprofile mit Zielvariable.
    """

    # --- 1. Daten preprocessen, damit alles numerisch vergleichbar ist ---
    X_processed = preprocessor.fit_transform(X_raw)

    # --- 2. Distanzmatrix berechnen (euklidischer Abstand) ---
    distances = pairwise_distances(X_processed, metric="euclidean")

    # --- 3. Erste Instanz zufällig wählen ---
    rng = np.random.default_rng(seed=random_state)
    first_idx = rng.integers(0, X_raw.shape[0])

    selected = [first_idx]

    # --- 4. Iterativ: immer den Punkt wählen, der den größten minimalen Abstand hat ---
    while len(selected) < n_profiles:
        # Für alle nicht gewählten Profile: Abstand zum nächsten bereits gewählten berechnen
        min_distances = np.min(distances[:, selected], axis=1)
        # Index des Profils mit größtem min. Abstand
        next_idx = np.argmax(min_distances)
        selected.append(next_idx)

    # --- 5. Originalprofile zurückgeben (mit Label) ---
    diverse_profiles = X_raw.iloc[selected].copy()
    diverse_profiles["Ziel"] = y.iloc[selected].values

    return diverse_profiles


# === Anwendung ===
diverse_profiles = select_diverse_profiles(X, y, preprocessor, n_profiles=10, random_state=42)

# Ergebnisse anzeigen
print("10 diverse Profile aus German Credit:")
print(diverse_profiles)


10 diverse Profile aus German Credit:
    Status_des_Girokontos  Dauer_in_Monaten Kreditgeschichte  \
89                    A11                12              A30   
915                   A12                48              A30   
186                   A12                 9              A31   
685                   A14                60              A32   
756                   A13                 6              A34   
653                   A12                36              A33   
344                   A13                10              A32   
175                   A14                30              A31   
255                   A12                60              A33   
846                   A14                18              A32   

    Kreditverwendungszweck  Kreditbetrag Sparkonto_Wertpapiere  \
89                     A45          1108                   A61   
915                   A410         18424                   A61   
186                    A41          5129                   

In [6]:

    # Kategoriewerte für jede Spalte definieren
category_mappings = {
  "Status_des_Girokontos": {
        "A11": "... < 0 DM",
        "A12": "0 <= ... < 200 DM",
        "A13": "... >= 200 DM / Gehaltspfändung mindestens 1 Jahr",
        "A14": "kein Girokonto"
    },

    "Dauer_in_Monaten": {},  # numerisch, daher keine Umkodierung erforderlich

    "Kreditgeschichte": {
        "A30": "keine Kredite aufgenommen / alle Kredite wurden ordnungsgemäß zurückgezahlt",
        "A31": "alle Kredite bei dieser Bank wurden ordnungsgemäß zurückgezahlt",
        "A32": "bestehende Kredite wurden bisher ordnungsgemäß zurückgezahlt",
        "A33": "Verzögerungen bei der Rückzahlung in der Vergangenheit",
        "A34": "kritisches Konto / andere Kredite bestehen (nicht bei dieser Bank)"
    },
    "Kreditverwendungszweck": {
        "A40": "Auto (neu)",
        "A41": "Auto (gebraucht)",
        "A42": "Möbel/Ausstattung",
        "A43": "Radio/TV",
        "A44": "Haushaltsgeräte",
        "A45": "Reparaturen",
        "A46": "Ausbildung",
        "A47": "Urlaub (existiert nicht?)",
        "A48": "Umschulung",
        "A49": "Geschäft",
        "A410": "Sonstiges"
    },
    "Kreditbetrag": {},  # numerisch, daher keine Umkodierung erforderlich

    "Sparkonto_Wertpapiere": {
        "A61": "... < 100 DM",
        "A62": "100 <= ... < 500 DM",
        "A63": "500 <= ... < 1000 DM",
        "A64": ".. >= 1000 DM",
        "A65": "unbekannt/kein Sparkonto"
    },
    "Beschäftigt_seit": {
        "A71": "arbeitslos",
        "A72": "... < 1 Jahr",
        "A73": "1 <= ... < 4 Jahre",
        "A74": "4 <= ... < 7 Jahre",
        "A75": ".. >= 7 Jahre"
    },
    "Ratenhöhe": {},  # numerisch, daher keine Umkodierung erforderlich

    "Familienstand_Geschlecht": {
        "A91": "männlich : geschieden/getrennt",
        "A92": "weiblich : geschieden/getrennt/verheiratet",
        "A93": "männlich : ledig",
        "A94": "männlich : verheiratet/witwer",
        "A95": "weiblich : ledig"
    },

    "Weitere_Bürgen_Schuldner": {
        "A101": "keine",
        "A102": "Mit-Antragsteller",
        "A103": "Bürge"
    },
    "Wohnsitzdauer": {},  # numerisch, daher keine Umkodierung erforderlich
    "Vermögen": {
        "A121": "Immobilie",
        "A122": "wenn nicht A121: Bausparvertrag/Lebensversicherung",
        "A123": "wenn nicht A121/A122: Auto oder anderes, nicht in Attribut 6",
        "A124": "unbekannt/kein Eigentum"
    },
    "Alter": {},  # numerisch, daher keine Umkodierung erforderlich
    "Andere_Ratenverpflichtungen": {
        "A141": "Bank",
        "A142": "Läden",
        "A143": "Keine"
    },
    "Wohnsituation": {
        "A151": "Miete",
        "A152": "Eigentum",
        "A153": "kostenlos"
    },
    "Anzahl_bestehender_Kredite": {},  # numerisch, daher keine Umkodierung erforderlich
    "Beruf": {
        "A171": "arbeitslos/ungelernt - nicht ansässig",
        "A172": "ungelernt - ansässig",
        "A173": "gelernt - Angestellter/Beamter",
        "A174": "Management/selbstständig/hochqualifizierter Angestellter/Beamter"
    },
    "Unterhaltspflichtige_Personen": {},  # numerisch, daher keine Umkodierung erforderlich
    "Telefon": {
        "A191": "kein Telefon",
        "A192": "ja, unter dem Namen des Kunden registriert"
    },
    "Ausländischer_Arbeiter": {
        "A201": "ja",
        "A202": "nein"
    }

}

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.base import clone

# ---------- 1) Auswahl balanciert & divers ----------
def select_diverse_balanced(X_raw, y, preprocessor, n_per_class=5, random_state=42):
    """
    Wählt pro Klasse (0/1) n_per_class möglichst diverse Profile (Max-Min innerhalb der Klasse).
    Gibt Originalprofile (mit Codes) + Ziel zurück.
    """
    rng = np.random.default_rng(random_state)
    parts = []

    for cls in [0, 1]:
        mask = (y == cls).values
        X_c = X_raw[mask]
        y_c = y[mask]

        # Preprocessing nur auf die Klassen-Teilmenge fitten (für Distanzraum)
        pre = clone(preprocessor)
        Xc_proc = pre.fit_transform(X_c)

        D = pairwise_distances(Xc_proc, metric="euclidean")

        first_idx = rng.integers(0, X_c.shape[0])
        selected = [first_idx]
        all_idx = np.arange(X_c.shape[0])

        while len(selected) < n_per_class:
            remaining = np.setdiff1d(all_idx, np.array(selected), assume_unique=True)
            min_d = D[remaining][:, selected].min(axis=1)
            next_idx = remaining[np.argmax(min_d)]
            selected.append(next_idx)

        block = X_c.iloc[selected].copy()
        block["Ziel"] = y_c.iloc[selected].values
        parts.append(block)

    out = pd.concat(parts, axis=0).reset_index(drop=True)
    return out

diverse_balanced = select_diverse_balanced(X, y, preprocessor, n_per_class=5, random_state=42)

# ---------- 2) Lesbare Ausgabe erzeugen ----------
def make_readable(df_profiles, categorical_cols, category_mappings, ziel_mapping={1: "guter Kredit", 0: "schlechter Kredit"}):
    """
    Mappt nur für die ANSICHT die Axx-Codes auf Klartext anhand category_mappings.
    Numerische Spalten bleiben unverändert.
    """


    readable = df_profiles.copy()

    # Ziel lesbar machen
    if "Ziel" in readable.columns:
        readable["Ziel"] = readable["Ziel"].map(ziel_mapping).fillna(readable["Ziel"])

    # Kategorische Spalten lesbar machen (falls es ein Mapping gibt)
    for col in categorical_cols:
        mapping = category_mappings.get(col, None)
        if mapping:
            readable[col] = readable[col].map(mapping).fillna(readable[col])

    # (Optional) Spaltenreihenfolge: zuerst Label, dann Rest
    cols = list(readable.columns)
    if "Ziel" in cols:
        cols = ["Ziel"] + [c for c in cols if c != "Ziel"]
        readable = readable[cols]

    return readable

readable_profiles = make_readable(diverse_balanced, categorical_cols, category_mappings)

# ---------- 3) Anzeigen & (optional) speichern ----------
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 140)

print("10 diverse Profile (balanciert) – lesbar:")
display(readable_profiles)  # in Colab/Jupyter

# Optional: CSV export für die Dokumentation
export_path = 'C:/Users/JonasNiehus/Documents/Masterarbeit/Evaluation/Ergebnisse/diverse_profiles.csv' 
readable_profiles.to_csv(export_path, index=False, encoding="utf-8")
print(f"\n✅ Lesbare Profile gespeichert unter: {export_path}")


10 diverse Profile (balanciert) – lesbar:


,Ziel,Status_des_Girokontos,Dauer_in_Monaten,Kreditgeschichte,Kreditverwendungszweck,Kreditbetrag,Sparkonto_Wertpapiere,Beschäftigt_seit,Ratenhöhe,Familienstand_Geschlecht,Weitere_Bürgen_Schuldner,Wohnsitzdauer,Vermögen,Alter,Andere_Ratenverpflichtungen,Wohnsituation,Anzahl_bestehender_Kredite,Beruf,Unterhaltspflichtige_Personen,Telefon,Ausländischer_Arbeiter
0,schlechter Kredit,kein Girokonto,18,alle Kredite bei dieser Bank wurden ordnungsgemäß zurückgezahlt,Auto (neu),6458,... < 100 DM,.. >= 7 Jahre,2,männlich : ledig,keine,4,unbekannt/kein Eigentum,39,Bank,Eigentum,2,Management/selbstständig/hochqualifizierter Angestellter/Beamter,2,"ja, unter dem Namen des Kunden registriert",ja
1,schlechter Kredit,0 <= ... < 200 DM,72,bestehende Kredite wurden bisher ordnungsgemäß zurückgezahlt,Radio/TV,5595,100 <= ... < 500 DM,1 <= ... < 4 Jahre,2,männlich : verheiratet/witwer,keine,2,"wenn nicht A121/A122: Auto oder anderes, nicht in Attribut 6",24,Keine,Eigentum,1,gelernt - Angestellter/Beamter,1,kein Telefon,ja
2,schlechter Kredit,0 <= ... < 200 DM,12,bestehende Kredite wurden bisher ordnungsgemäß zurückgezahlt,Möbel/Ausstattung,951,100 <= ... < 500 DM,... < 1 Jahr,4,weiblich : geschieden/getrennt/verheiratet,keine,4,"wenn nicht A121/A122: Auto oder anderes, nicht in Attribut 6",27,Bank,Miete,4,gelernt - Angestellter/Beamter,1,kein Telefon,ja
3,schlechter Kredit,... >= 200 DM / Gehaltspfändung mindestens 1 Jahr,12,alle Kredite bei dieser Bank wurden ordnungsgemäß zurückgezahlt,Geschäft,609,... < 100 DM,... < 1 Jahr,4,weiblich : geschieden/getrennt/verheiratet,keine,1,Immobilie,26,Keine,Eigentum,1,arbeitslos/ungelernt - nicht ansässig,1,kein Telefon,ja
4,schlechter Kredit,0 <= ... < 200 DM,54,keine Kredite aufgenommen / alle Kredite wurden ordnungsgemäß zurückgezahlt,Geschäft,15945,... < 100 DM,... < 1 Jahr,3,männlich : ledig,keine,4,unbekannt/kein Eigentum,58,Keine,Miete,1,gelernt - Angestellter/Beamter,1,"ja, unter dem Namen des Kunden registriert",ja
5,guter Kredit,kein Girokonto,24,kritisches Konto / andere Kredite bestehen (nicht bei dieser Bank),Radio/TV,2223,100 <= ... < 500 DM,.. >= 7 Jahre,4,männlich : ledig,keine,4,wenn nicht A121: Bausparvertrag/Lebensversicherung,52,Bank,Eigentum,2,gelernt - Angestellter/Beamter,1,kein Telefon,ja
6,guter Kredit,kein Girokonto,60,Verzögerungen bei der Rückzahlung in der Vergangenheit,Radio/TV,15653,... < 100 DM,4 <= ... < 7 Jahre,2,männlich : ledig,keine,4,"wenn nicht A121/A122: Auto oder anderes, nicht in Attribut 6",21,Keine,Eigentum,2,gelernt - Angestellter/Beamter,1,"ja, unter dem Namen des Kunden registriert",ja
7,guter Kredit,... >= 200 DM / Gehaltspfändung mindestens 1 Jahr,6,kritisches Konto / andere Kredite bestehen (nicht bei dieser Bank),Auto (neu),1299,... < 100 DM,1 <= ... < 4 Jahre,1,männlich : ledig,keine,1,Immobilie,74,Keine,Eigentum,3,arbeitslos/ungelernt - nicht ansässig,2,kein Telefon,nein
8,guter Kredit,kein Girokonto,4,bestehende Kredite wurden bisher ordnungsgemäß zurückgezahlt,Möbel/Ausstattung,601,... < 100 DM,... < 1 Jahr,1,weiblich : geschieden/getrennt/verheiratet,keine,3,Immobilie,23,Keine,Miete,1,ungelernt - ansässig,2,kein Telefon,ja
9,guter Kredit,0 <= ... < 200 DM,42,alle Kredite bei dieser Bank wurden ordnungsgemäß zurückgezahlt,Auto (gebraucht),9283,... < 100 DM,arbeitslos,1,männlich : ledig,keine,2,unbekannt/kein Eigentum,55,Bank,kostenlos,1,Management/selbstständig/hochqualifizierter Angestellter/Beamter,1,"ja, unter dem Namen des Kunden registriert",ja



✅ Lesbare Profile gespeichert unter: C:/Users/JonasNiehus/Documents/Masterarbeit/Evaluation/Ergebnisse/diverse_profiles.csv
